In [1]:
%tensorflow_version 2.x
!pip install -q gpt-2-simple
!pip install transformers
from google.colab import files
from datetime import datetime
import gpt_2_simple as gpt2
import pandas as pd
from transformers import GPT2Tokenizer

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.5 MB/s eta 0:00:00


**Downloading GPT2 124M model**

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 527Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.74Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 568Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:21, 23.4Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 574Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.15Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.15Mit/s]


**Mounting google drive and copying the training file**

In [2]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [4]:
file_name = "train.txt"

In [5]:
# gpt2.copy_file_from_gdrive(file_name)

**Finetuning GPT2 model for our Trainset**

In [6]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=10000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.30s/it]


dataset has 525420 tokens
Training...
[10 | 11.79] loss=3.09 avg=3.09
[20 | 14.97] loss=2.69 avg=2.89
[30 | 18.16] loss=2.77 avg=2.85
[40 | 21.36] loss=2.76 avg=2.83
[50 | 24.53] loss=2.39 avg=2.74
[60 | 27.71] loss=2.46 avg=2.69
[70 | 30.88] loss=2.56 avg=2.67
[80 | 34.08] loss=2.67 avg=2.67
[90 | 37.25] loss=2.40 avg=2.64
[100 | 40.42] loss=2.57 avg=2.63
[110 | 43.59] loss=2.64 avg=2.63
[120 | 46.78] loss=2.20 avg=2.60
[130 | 49.97] loss=2.18 avg=2.56
[140 | 53.14] loss=2.42 avg=2.55
[150 | 56.32] loss=2.36 avg=2.54
[160 | 59.50] loss=2.40 avg=2.53
[170 | 62.67] loss=2.31 avg=2.51
[180 | 65.85] loss=2.19 avg=2.50
[190 | 69.03] loss=2.44 avg=2.49
[200 | 72.22] loss=2.38 avg=2.49
======== SAMPLE 1 ========
 do not tolerate unacceptable conduct with others.
    In what capacity may this be normal? Is the supervisor appropriate? If not, what makes this behavior so unacceptable? What are you looking for other than the supervisor to respect? Are you actively seeking out ways to reduce your

Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========
 what of ourselves, what he would or would not do, what he sees, feels, and how God uses his word. These questions are not easy to answer, but as he becomes a minister and works with others who follow the scripture, you'll start to get a sense for what a minister is like. <|endoftext|>
question: There are so many issues facing this family. He is a minister. My brother is a minister. My sister is a minister. We have never met, never will meet. I feel like he is missing something important. He mentioned he may be gay, but is it trans? I want answers; I need them in a way that they don't come across as fake. He said he may be transgender, but is it real? context: Your questions demonstrate your concern for your safety and well-being as well as your mental and physical well-being.  I would be curious with what you feel is the social context in which you live.  I see a spectrum: those who do not wish to be identified as such are at increased risk for depression, a

**Saving model to drive**

In [7]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [3]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [4]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-10000


**TESTING**

In [7]:
test_data = pd.read_csv("test.csv")
queries = list(test_data["input_text"])
responses = list(test_data["target_text"])


In [8]:
# gpt2.generate(sess, run_name='run1',temperature=0.7,
#               nsamples=50,
#               batch_size=50)

In [16]:
generated_responses=[]
for i in queries:
  pref = "question: "+i+" context:"
  gpt2.generate_to_file(sess,
                destination_path="out.txt",
                prefix= pref,
                top_k=10, 
                top_p=0.7,
                temperature = 0.8,
                nsamples=1,
                batch_size=1,
                run_name='run1',
                truncate=' <|endoftext|>',
                include_prefix=False,
                )
  out_file = open("out.txt","r+")
  generated_responses.append(out_file.read()[:-2])
  print(len(generated_responses))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


**Metrics**

In [17]:
!pip install "nltk==3.4.5"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449923 sha256=bf4d45992a07d1575149b17cb2e9ede94bf273516f538b6e1b1c69726c25b817
  Stored in directory: /root/.cache/pip/wheels/04/32/57/69e42ad50941013def31e288c6e06bb569442dd993a123cb76
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [18]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist


In [19]:
def get_metrics(pred, target):
    turns = len(target)
    bleu_2 = 0
    bleu_4 = 0
    meteor = 0
    nist_2 = 0
    nist_4 = 0
    for index in range(turns):
        pred_utt = pred[index]
        target_utt = target[index]
        min_len = min(len(pred_utt), len(target_utt))
        lens = min(min_len, 4)
        if lens == 0:
            continue
        if lens >= 4:
            bleu_4_utt = sentence_bleu([target_utt], pred_utt, weights = (0.25, 0.25, 0.25, 0.25), smoothing_function = SmoothingFunction().method1)
            nist_4_utt = sentence_nist([target_utt], pred_utt, 4)
        else:
            bleu_4_utt = 0
            nist_4_utt = 0
        if lens >= 2:
            bleu_2_utt = sentence_bleu([target_utt], pred_utt, weights = (0.5, 0.5), smoothing_function = SmoothingFunction().method1)
            nist_2_utt = sentence_nist([target_utt], pred_utt, 2)
        else:
            bleu_2_utt = 0
            nist_2_utt = 0
            
        bleu_2 += bleu_2_utt
        bleu_4 += bleu_4_utt
        meteor += meteor_score([" ".join(target_utt)], " ".join(pred_utt))
        nist_2 += nist_2_utt
        nist_4 += nist_4_utt
        
    bleu_2 /= turns
    bleu_4 /= turns
    meteor /= turns
    nist_2 /= turns
    nist_4 /= turns
    return bleu_2, bleu_4, meteor, nist_2, nist_4


In [20]:
nltk.download('wordnet')
bleu_2, bleu_4, meteor, nist_2, nist_4 = get_metrics(generated_responses, responses)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:
print("bleu_2",bleu_2)
print("bleu_4",bleu_4)
print("meteor",meteor)
print("nist_2",nist_2)
print("nist_4",nist_4)

bleu_2 0.4451666503993174
bleu_4 0.2939982596113935
meteor 0.3861091928395942
nist_2 3.053938345854437
nist_4 3.4386304341956144


**Interactive Chat**

In [23]:
for step in range(1):
  inp = input("User:\n")
  pref = "question: "+inp+" context:"
  print("Chatbot:")
  gpt2.generate(sess,
                prefix= pref,
                top_k=10, 
                top_p=0.7,
                temperature = 0.8,
                nsamples=1,
                batch_size=1,
                run_name='run1',
                truncate=' <|endoftext|>',
                include_prefix=False,
                )

User:
I just feel sad all the time and I don't like anyone in my family. I feel like they're trying to control me and won't let me grow.
Chatbot:
 y'know, the good news is that people are starting to see that a little bit more clearly...that is, more than you realize and it's giving you a lot of room to grow.Lulling yourself with self-doubt is a great way to get at the root of it, the underlying fear is still underneath the surface fear, to gain some stability and acceptance.  I call this "depression." It's a mood ailment.  Feelings like anger, shame, guilt, fear are not-so-good motivation.Depression is a mood ailment and it can manifest in many ways. I'd recommend that you focus on dreams, listen to your dreams, ask your partner what he or she does during these dreams, then you'll have a better chance of finding the motivation to stay in the relationship.If you sleep at night, panic attacks and  feeling like you are going crazy- just because you are not going to sleep at all does not 